In [1]:
# import sportsref
# import os, csv, sqlite3
import pandas as pd
import numpy as np
import time
from datetime import datetime

# importing csvs to dataframes

In [2]:
# list of dataframes
gmin = 'gameInfo'
inst = 'injuryStatus'
psdr = 'passDirections'
snct = 'snapCounts'
strt = 'starters'
sdef = 'statsDefense'
skck = 'statsKicking'
soff = 'statsOffense'
srtn = 'statsReturns'
stm  = 'statsTeam'
tmrs = 'teamRoster'

dfl = [gmin, inst, psdr, snct, strt, sdef, skck, soff, srtn, stm, tmrs]

In [3]:
dfIn = {
    'gameInfo'      :['season','week','bsID'                  ],
    'injuryStatus'  :['season','week',       'team','playerID'],
    'passDirections':['season','week','bsID','team','playerID'],
    'snapCounts'    :['season','week','bsID','team','playerID'],
    'starters'      :['season','week','bsID','team','playerID'],
    'statsDefense'  :['season','week','bsID','team','playerID'],
    'statsKicking'  :['season','week','bsID','team','playerID'],
    'statsOffense'  :['season','week','bsID','team','playerID'],
    'statsReturns'  :['season','week','bsID','team','playerID'],
    'statsTeam'     :['season','week','bsID','team'           ],
    'teamRoster'    :['season',              'team','playerID'],
}

In [1]:
# import csvs into dataframes
# directory = '/Users/phil/Google Drive/nfl/data'
directory = 'C:/Users/phil/Google Drive/nfl/data'
# directory = 'C:/Users/lyncp010/projects/nfl/data'

d = {}
for key in dfIn:
    d[key] = pd.read_csv(directory + '/{}_07_15.csv'.format(key), low_memory=False)
    d[key].set_index(dfIn[key], inplace=True)
    d[key].sortlevel(inplace=True)
print 'tables imported'

NameError: name 'dfIn' is not defined

#### fix roster

In [13]:
# fixing roster experience
d[tmrs].ix[d[tmrs]['experience']=='Rook', 'experience'] = '0'
d[tmrs]['experience'] = d[tmrs]['experience'].astype(float)

#### fix team stats

In [14]:
idx = pd.IndexSlice
homeAwayOpp = {True:'away', False:'home'}

for i, r in d[stm].iterrows():
    print '\r{}'.format(i),
    seas = i[0]
    week = i[1]
    bsid = i[2]
    team = i[3]
    d[stm].loc[i, 'opponent'] = d[gmin].loc[idx[[seas],[week],[bsid]],][homeAwayOpp[r['home']]][0] #foo[homeAwayOpp[r['home']]][0]
    foo = d[stm].loc[idx[[seas],[week],[bsid],d[stm].loc[i, 'opponent']],]
    d[stm].loc[i, 'defPassCmpAlw'] = foo['passCmp'][0]
    d[stm].loc[i, 'defPassYdsAlw'] = foo['passYds'][0]
    d[stm].loc[i, 'defPassTdsAlw'] = foo['passTds'][0]
    d[stm].loc[i, 'defRushYdsAlw'] = foo['rushYds'][0]
    d[stm].loc[i, 'defRushTdsAlw'] = foo['rushTds'][0]
    d[stm].loc[i, 'defTotalYardsAlw'] = foo['totalYards'][0]
    d[stm].loc[i, 'defFirstDownsAlw'] = foo['firstDowns'][0]
    d[stm].loc[i, 'defPassInt'] = foo['passInt'][0]
    d[stm].loc[i, 'defSacks'] = foo['sacks'][0]
    d[stm].loc[i, 'defSacksYds'] = foo['sacksYds'][0]
    d[stm].loc[i, 'defFFumbles'] = foo['fumbles'][0]
    
d[stm] = d[stm].rename(columns = {
        'passAtt':'offPassAtt',
        'passCmp':'offPassCmp',
        'passYds':'offPassYds',
        'netPassYards':'offNetPassYards',
        'passTds':'offPassTds',
        'rushAtt':'offRushAtt',
        'rushYds':'offRushYds',
        'rushTds':'offRushTds',
        'totalYards':'offTotalYards',
        'firstDowns':'offFirstDowns',
        'sacks':'offSacks',
        'sacksYds':'offSacksYds',
        'passInt':'offPassInt',
        'fumbles':'offFumbles',
        'fumblesLost':'offFumblesLost',
        'turnovers':'offTurnovers',
        'thirdDownAtt':'offThirdDownAtt',
        'thirdDownConv':'offThirdDownConv',
        'fourthDownAtt':'offFourthDownAtt',
        'fourthDownConv':'offFourthDownConv'
    })

# fixing time
d[stm]['timeOfPossession'] = d[stm]['timeOfPossession'].apply(lambda x: float(x.split(':')[0]) + float(x.split(':')[1])/60)

# fixing column order
cols = ['home', 'opponent']
cols += [c for c in d[stm].columns if ('off' in c)]
cols += [c for c in d[stm].columns if ('def' in c)]
cols += ['pentalties', 'pentaltiesYds', 'timeOfPossession']
d[stm] = d[stm][cols]

(2015L, 21L, '201602070den', 'den')


In [15]:
# print data frames with indexes + columns
for df in sorted(dfl):
    print df + '\n'+'-'*len(df)
    for i in d[df].index.names:
        print '* {}'.format(i)
    print d[df].dtypes
    print '\n'

gameInfo
--------
* season
* week
* bsID
date            object
weekday         object
startTime       object
home            object
away            object
winner          object
homeScore        int64
awayScore        int64
line           float64
overUnder      float64
roof            object
surface         object
temp             int64
relHumidity    float64
windChill        int64
windMPH          int64
dtype: object


injuryStatus
------------
* season
* week
* team
* playerID
status        object
didNotPlay      bool
dtype: object


passDirections
--------------
* season
* week
* bsID
* team
* playerID
rec_catches_d         float64
rec_catches_dl        float64
rec_catches_dm        float64
rec_catches_dr        float64
rec_catches_s         float64
rec_catches_sl        float64
rec_catches_sm        float64
rec_catches_sr        float64
rec_targets_d         float64
rec_targets_dl        float64
rec_targets_dm        float64
rec_targets_dr        float64
rec_targets_s         floa

# testing

In [ ]:
idx = pd.IndexSlice

In [ ]:
d[inst].head(3)

In [ ]:
d[inst].loc[idx[[2008],:,:,:],]#['didNotPlay']#.mean()

In [ ]:
d[snct].head(3)

In [ ]:
d[inst].loc[idx[[2009],1,['nyg'],:],]#['didNotPlay']#.mean()

In [ ]:
# (2009L, 1L, '200909100pit', 'oti', 'CollKe00')

In [ ]:
d[snct].index.dtype

In [ ]:
d[snct].loc[idx[[2009],:,:,'oti',:],]#['offPct'].mean()

In [22]:
start = time.time()
idx = pd.IndexSlice

# adding features to quarterback df
# qbdf = qdf(d[strt], seas=[2015], week=range(1,22))
qbdf = d[strt].loc[idx[[2015],range(1,22)],]
qbdf = qbdf[qbdf.position == 'QB']

homeAwayOpp = {True:'away', False:'home'}
first_season = 2007

for i, r in qbdf.iterrows():
    print '\r{}'.format(i),
    #print 'seas={}, week={}, bsid={}, team={}, plid={}'.format(i[0], i[1], i[2], i[3], i[4])
    seas = i[0]
    week = i[1]
    bsid = i[2]
    team = i[3]
    plid = i[4]
    
    # fantasy points
    foo = d[soff].loc[idx[[seas],[week],:,:,[plid]],]
    qbdf.loc[i, 'ff_PassTds'] = foo['passTds'].mean()
    qbdf.loc[i, 'ff_PassYds'] = foo['passYds'].mean()
    qbdf.loc[i, 'ff_PassYdsOver300'] = float(foo['passYds'].mean() > 300)
    qbdf.loc[i, 'ff_Int'] = foo['passInt'].mean()
    qbdf.loc[i, 'ff_RushYds'] = foo['rushYds'].mean()
    qbdf.loc[i, 'ff_RushTds'] = foo['rushTd'].mean()
    qbdf.loc[i, 'ff_RushYdsOver100'] = float(foo['rushYds'].mean() > 100)
    qbdf.loc[i, 'ff_Fumble'] = foo['fumbles'].mean()
    qbdf.loc[i, 'ff_RecYds'] = foo['recYds'].mean()
    qbdf.loc[i, 'ff_Recs'] = foo['rec'].mean()
    qbdf.loc[i, 'ff_RecTds'] = foo['recTds'].mean()
    qbdf.loc[i, 'ff_RecYdsOver100'] = float(foo['recYds'].mean() > 100)
    
    # team roster - player stats
    foo = d[tmrs].loc[idx[[seas],[team],[plid]],]
    qbdf.loc[i, 'playerAge'] = foo['age'].mean()
    qbdf.loc[i, 'playerWeight'] = foo['weight'].mean()
    qbdf.loc[i, 'pfrApproxValue'] = foo['pfrApproxValue'].mean()
    qbdf.loc[i, 'experience'] = foo['experience'].mean()
    
    # injury status
    foo = d[inst].loc[idx[[seas],[week],[team],[plid]],]
    if len(foo)==1:
        qbdf.loc[i, 'injuryStatus'] = foo['status'][0]
        qbdf.loc[i, 'didNotPlay'] = foo['didNotPlay'][0]
    else:
        qbdf.loc[i, 'injuryStatus'] = 'None'
        qbdf.loc[i, 'didNotPlay'] = False
    
    # snap counts
    qbdf.loc[i, 'snapCountPct'] = d[snct].loc[idx[[seas],[week],[bsid],team,plid],]['offPct'].mean()
    
    # game info
    foo = d[gmin].loc[idx[[seas],[week],[bsid]],]
    qbdf.loc[i, 'roof'] = foo['roof'][0]
    qbdf.loc[i, 'opponent'] = foo[homeAwayOpp[r['home']]][0]
    qbdf.loc[i, 'startTime'] = foo['startTime'][0]
    qbdf.loc[i, 'temp'] = foo['temp'][0]
    qbdf.loc[i, 'weekday'] = foo['weekday'][0]
    qbdf.loc[i, 'windChill'] = foo['windChill'][0]
    qbdf.loc[i, 'windMPH'] = foo['windMPH'][0]
    if week==1:
        qbdf.loc[i, 'restDays'] = 14.0
    else:
        df = d[gmin].loc[idx[[seas],[week-1,week-2],:],]
        gameDate = datetime.strptime(foo['date'][0], '%Y-%m-%d')
        lastGameDate = datetime.strptime(df[(df['home']==team) | (df['away']==team)]['date'][-1], '%Y-%m-%d')
        qbdf.loc[i, 'restDays'] = (gameDate - lastGameDate).days
    
    # qb passing stats - prev years
    foo = d[soff].loc[idx[[seas-1],:,:,:,[plid]],]
    qbdf.loc[i, 'prev1SeasPassCmpAvg'] = foo['passCmp'].mean()
    qbdf.loc[i, 'prev1SeasPassAttAvg'] = foo['passAtt'].mean()
    qbdf.loc[i, 'prev1SeasPassYdAvg'] = foo['passYds'].mean()
    qbdf.loc[i, 'prev1SeasPassTdAvg'] = foo['passTds'].mean()
    qbdf.loc[i, 'prev1SeasPassIntAvg'] = foo['passInt'].mean()
    qbdf.loc[i, 'prev1SeasPassRtngAvg'] = foo['passRating'].mean()  
    
    foo = d[soff].loc[idx[[seas-2],:,:,:,[plid]],]
    qbdf.loc[i, 'prev2SeasPassCmpAvg'] = foo['passCmp'].mean()
    qbdf.loc[i, 'prev2SeasPassAttAvg'] = foo['passAtt'].mean()
    qbdf.loc[i, 'prev2SeasPassYdAvg'] = foo['passYds'].mean()
    qbdf.loc[i, 'prev2SeasPassTdAvg'] = foo['passTds'].mean()
    qbdf.loc[i, 'prev2SeasPassIntAvg'] = foo['passInt'].mean()
    qbdf.loc[i, 'prev2SeasPassRtngAvg'] = foo['passRating'].mean()
    
    foo = d[soff].loc[idx[range(first_season, seas),:,:,:,[plid]],]
    qbdf.loc[i, 'prevASeasPassCmpAvg'] = foo['passCmp'].mean()
    qbdf.loc[i, 'prevASeasPassAttAvg'] = foo['passAtt'].mean()
    qbdf.loc[i, 'prevASeasPassYdAvg'] = foo['passYds'].mean()
    qbdf.loc[i, 'prevASeasPassTdAvg'] = foo['passTds'].mean()
    qbdf.loc[i, 'prevASeasPassIntAvg'] = foo['passInt'].mean()
    qbdf.loc[i, 'prevASeasPassRtngAvg'] = foo['passRating'].mean()
    
    # qb passing stats - current year
    foo = d[soff].loc[idx[[seas],range(1,week),:,:,[plid]],]
    qbdf.loc[i, 'curSeasPassCmpAvg'] = foo['passCmp'].mean()
    qbdf.loc[i, 'curSeasPassCmpMin'] = foo['passCmp'].min()
    qbdf.loc[i, 'curSeasPassCmpMax'] = foo['passCmp'].max()
    qbdf.loc[i, 'curSeasPassAttAvg'] = foo['passAtt'].mean()
    qbdf.loc[i, 'curSeasPassAttMin'] = foo['passAtt'].min()
    qbdf.loc[i, 'curSeasPassAttMax'] = foo['passAtt'].max()
    qbdf.loc[i, 'curSeasPassYdAvg'] = foo['passYds'].mean()
    qbdf.loc[i, 'curSeasPassYdMin'] = foo['passYds'].min()
    qbdf.loc[i, 'curSeasPassYdMax'] = foo['passYds'].max()
    qbdf.loc[i, 'curSeasPassTdAvg'] = foo['passTds'].mean()
    qbdf.loc[i, 'curSeasPassTdMin'] = foo['passTds'].min()
    qbdf.loc[i, 'curSeasPassTdMax'] = foo['passTds'].max()
    qbdf.loc[i, 'curSeasPassIntAvg'] = foo['passInt'].mean()
    qbdf.loc[i, 'curSeasPassIntMin'] = foo['passInt'].min()
    qbdf.loc[i, 'curSeasPassIntMax'] = foo['passInt'].max()
    qbdf.loc[i, 'curSeasPassSackAvg'] = foo['passSacked'].mean()
    qbdf.loc[i, 'curSeasPassSackMin'] = foo['passSacked'].min()
    qbdf.loc[i, 'curSeasPassSackMax'] = foo['passSacked'].max()
    qbdf.loc[i, 'curSeasPassLongAvg'] = foo['passLong'].mean()
    qbdf.loc[i, 'curSeasPassLongMin'] = foo['passLong'].min()
    qbdf.loc[i, 'curSeasPassLongMax'] = foo['passLong'].max()
    qbdf.loc[i, 'curSeasPassRtngAvg'] = foo['passRating'].mean()
    qbdf.loc[i, 'curSeasPassRtngMin'] = foo['passRating'].min()
    qbdf.loc[i, 'curSeasPassRtngMax'] = foo['passRating'].max()
    
    # opp deffense sack, int, and ffum - sum starting def players stats for each week
    defStarters = d[sdef].loc[idx[[seas],[week],[bsid],[qbdf.loc[i, 'opponent']],:],].index.get_level_values('playerID').unique()
    if week==1:
        qbdf.loc[i, 'curSeasDefSackAvg'] = np.nan
        qbdf.loc[i, 'curSeasDefSackMin'] = np.nan
        qbdf.loc[i, 'curSeasDefSackMax'] = np.nan
        qbdf.loc[i, 'curSeasDefIntAvg'] = np.nan
        qbdf.loc[i, 'curSeasDefIntMin'] = np.nan
        qbdf.loc[i, 'curSeasDefIntMax'] = np.nan
        qbdf.loc[i, 'curSeasDefFFumAvg'] = np.nan
        qbdf.loc[i, 'curSeasDefFFumMin'] = np.nan
        qbdf.loc[i, 'curSeasDefFFumMax'] = np.nan
    else:
        foo = d[sdef].loc[idx[[seas],range(1,week),:,[qbdf.loc[i, 'opponent']],:],]
        qbdf.loc[i, 'curSeasDefSackAvg'] = np.mean([foo['sacks'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefSackMin'] = min([foo['sacks'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefSackMax'] = max([foo['sacks'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefIntAvg'] = np.mean([foo['def_int'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefIntMin'] = min([foo['def_int'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefIntMax'] = max([foo['def_int'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefFFumAvg'] = np.mean([foo['fumbles_forced'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefFFumMin'] = min([foo['fumbles_forced'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefFFumMax'] = max([foo['fumbles_forced'].sum() for wk in range(1,week)])
    
    # team stats for each week
    foo = d[stm].loc[idx[[seas],range(1,week),:,[team]],]
    qbdf.loc[i, 'curSeasOffSackAvg'] = foo['offSacks'].mean()
    qbdf.loc[i, 'curSeasOffFirstDownsAvg'] = foo['offFirstDowns'].mean()
    qbdf.loc[i, 'curSeasOffFourthDownAttAvg'] = foo['offFourthDownAtt'].mean()
    qbdf.loc[i, 'curSeasOffFourthDownConvAvg'] = foo['offFourthDownConv'].mean()
    
print '\nDone! Total run time: {}'.format(time.time() - start)
# qbdf.tail()

(2015L, 21L, '201602070den', 'den', 'MannPe00') 
Done! Total run time: 57.0590000153


In [24]:
qbdf.tail()

# starters

In [ ]:
season = 2015
starters = {}

# 2015 offensive starters
df = d[strt].loc[season]
for pos in ['QB','RB','WR','TE']:
    starters[pos] = df[df.position==pos].index.get_level_values('playerID').unique()

# kicker starters
df = d[skck].ix[season]
starters['K'] = df[(~np.isnan(df.xpa)) | (np.isnan(df.fga))].index.get_level_values('playerID').unique()

starters

# appending stats

In [ ]:
# query stats dataframe by index
def qdf(df, seas=True, week=True, bsid=True, team=True, plid=True):
    if not type(seas)==bool: seas = (df.index.get_level_values('season').isin(seas))
    if not type(week)==bool: week = (df.index.get_level_values('week').isin(week))
    if not type(bsid)==bool: bsid = (df.index.get_level_values('bsID').isin(bsid))
    if not type(team)==bool: team = (df.index.get_level_values('team').isin(team))
    if not type(plid)==bool: plid = (df.index.get_level_values('playerID').isin(plid))
    return df[seas & week & week & bsid & team & plid]

#### quarterbacks

In [ ]:
# adding features to quarterback df
qbdf = qdf(d[strt], seas=[2009, 2010, 2011, 2012, 2013, 2014, 2015]) #, week=[1])
qbdf = qbdf[qbdf.position == 'QB']

homeAwayOpp = {True:'away', False:'home'}

for i, r in qbdf.iterrows():
    # print i, x['home']
    #print 'seas={}, week={}, bsid={}, team={}, plid={}'.format(i[0], i[1], i[2], i[3], i[4])
    seas = i[0]
    week = i[1]
    bsid = i[2]
    team = i[3]
    plid = i[4]
    
    # fantasy points
    qbdf.loc[i, 'playerID'] = plid
    qbdf.loc[i, 'ff_PassTds'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['passTds'].mean()
    qbdf.loc[i, 'ff_PassYds'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['passYds'].mean()
    qbdf.loc[i, 'ff_PassYdsOver300'] = float(qdf(d[soff], seas=[seas], week=[week], plid=[plid])['passYds'].mean() > 300)
    qbdf.loc[i, 'ff_Int'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['passInt'].mean()
    qbdf.loc[i, 'ff_RushYds'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['rushYds'].mean()
    qbdf.loc[i, 'ff_RushTds'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['rushTd'].mean()
    qbdf.loc[i, 'ff_RushYdsOver100'] = float(qdf(d[soff], seas=[seas], week=[week], plid=[plid])['rushYds'].mean() > 100)
    qbdf.loc[i, 'ff_Fumble'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['fumbles'].mean()
    qbdf.loc[i, 'ff_RecYds'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['recYds'].mean()
    qbdf.loc[i, 'ff_Recs'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['rec'].mean()
    qbdf.loc[i, 'ff_RecTds'] = qdf(d[soff], seas=[seas], week=[week], plid=[plid])['recTds'].mean()
    qbdf.loc[i, 'ff_RecYdsOver100'] = float(qdf(d[soff], seas=[seas], week=[week], plid=[plid])['recYds'].mean() > 100)
    
    # team roster - player stats
#     qbdf.loc[i, 'playerAge'] = qdf(d[tmrs], seas=[seas], plid=[plid])['age'].mean()
#     qbdf.loc[i, 'playerWeight'] = qdf(d[tmrs], seas=[seas], plid=[plid])['weight'].mean()
    qbdf.loc[i, 'pfrApproxValue'] = qdf(d[tmrs], seas=[seas], plid=[plid])['pfrApproxValue'].mean()
#     qbdf.loc[i, 'experience'] = qdf(d[tmrs], seas=[seas], plid=[plid])['experience']
    
    # injury status
    try:
        qbdf.loc[i, 'injuryStatus'] = qdf(d[inst], seas=[seas], week=[week], plid=[plid])['status'][0]
        qbdf.loc[i, 'didNotPlay'] = qdf(d[inst], seas=[seas], week=[week], plid=[plid])['didNotPlay'][0]
    except IndexError:
        qbdf.loc[i, 'injuryStatus'] = 'None'
        qbdf.loc[i, 'didNotPlay'] = False
    
    # snap counts
    qbdf.loc[i, 'snapCountPct'] = qdf(d[snct], seas=[seas], bsid=[bsid], plid=[plid])['offPct'].mean()
    
    # game info
#     qbdf.loc[i, 'roof'] = qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])['roof'][0]
    qbdf.loc[i, 'opponent'] = qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])[homeAwayOpp[r['home']]][0]
    qbdf.loc[i, 'startTime'] = qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])['startTime'][0]
    qbdf.loc[i, 'temp'] = qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])['temp'][0]
    qbdf.loc[i, 'weekday'] = qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])['weekday'][0]
    qbdf.loc[i, 'windChill'] = qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])['windChill'][0]
    qbdf.loc[i, 'windMPH'] = qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])['windMPH'][0]
    if week==1:
        qbdf.loc[i, 'restDays'] = np.nan
    else:
        df = qdf(d[gmin], seas=[seas], week=[week-1,week-2])
        gameDate = datetime.strptime(qdf(d[gmin], seas=[seas], week=[week], bsid=[bsid])['date'][0], '%Y-%m-%d')
        lastGameDate = datetime.strptime(df[(df['home']==team) | (df['away']==team)]['date'][-1], '%Y-%m-%d')
        qbdf.loc[i, 'restDays'] = (gameDate - lastGameDate).days
    
    # qb passing stats - prev years
    first_season = 2007
    qbdf.loc[i, 'prev1SeasPassCmpAvg'] = qdf(d[soff], seas=[seas-1], plid=[plid])['passCmp'].mean()
    qbdf.loc[i, 'prev2SeasPassCmpAvg'] = qdf(d[soff], seas=[seas-2], plid=[plid])['passCmp'].mean()
    qbdf.loc[i, 'prevASeasPassCmpAvg'] = qdf(d[soff], seas=range(first_season, seas), plid=[plid])['passCmp'].mean()
    qbdf.loc[i, 'prev1SeasPassAttAvg'] = qdf(d[soff], seas=[seas-1], plid=[plid])['passAtt'].mean()
    qbdf.loc[i, 'prev2SeasPassAttAvg'] = qdf(d[soff], seas=[seas-2], plid=[plid])['passAtt'].mean()
    qbdf.loc[i, 'prevASeasPassAttAvg'] = qdf(d[soff], seas=range(first_season, seas), plid=[plid])['passAtt'].mean()
    qbdf.loc[i, 'prev1SeasPassYdAvg'] = qdf(d[soff], seas=[seas-1], plid=[plid])['passYds'].mean()
    qbdf.loc[i, 'prev2SeasPassYdAvg'] = qdf(d[soff], seas=[seas-2], plid=[plid])['passYds'].mean()
    qbdf.loc[i, 'prevASeasPassYdAvg'] = qdf(d[soff], seas=range(first_season, seas), plid=[plid])['passYds'].mean()
    qbdf.loc[i, 'prev1SeasPassTdAvg'] = qdf(d[soff], seas=[seas-1], plid=[plid])['passTds'].mean()
    qbdf.loc[i, 'prev2SeasPassTdAvg'] = qdf(d[soff], seas=[seas-2], plid=[plid])['passTds'].mean()
    qbdf.loc[i, 'prevASeasPassTdAvg'] = qdf(d[soff], seas=range(first_season, seas), plid=[plid])['passTds'].mean()
    qbdf.loc[i, 'prev1SeasPassIntAvg'] = qdf(d[soff], seas=[seas-1], plid=[plid])['passInt'].mean()
    qbdf.loc[i, 'prev2SeasPassIntAvg'] = qdf(d[soff], seas=[seas-2], plid=[plid])['passInt'].mean()
    qbdf.loc[i, 'prevASeasPassIntAvg'] = qdf(d[soff], seas=range(first_season, seas), plid=[plid])['passInt'].mean()
    qbdf.loc[i, 'prev1SeasPassRtngAvg'] = qdf(d[soff], seas=[seas-1], plid=[plid])['passRating'].mean()
    qbdf.loc[i, 'prev2SeasPassRtngAvg'] = qdf(d[soff], seas=[seas-2], plid=[plid])['passRating'].mean()
    qbdf.loc[i, 'prevASeasPassRtngAvg'] = qdf(d[soff], seas=range(first_season, seas), plid=[plid])['passRating'].mean()
    
    # qb passing stats - current year
    qbdf.loc[i, 'curSeasPassCmpAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passCmp'].mean()
    qbdf.loc[i, 'curSeasPassCmpMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passCmp'].min()
    qbdf.loc[i, 'curSeasPassCmpMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passCmp'].max()
    qbdf.loc[i, 'curSeasPassAttAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passAtt'].mean()
    qbdf.loc[i, 'curSeasPassAttMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passAtt'].min()
    qbdf.loc[i, 'curSeasPassAttMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passAtt'].max()
    qbdf.loc[i, 'curSeasPassYdAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passYds'].mean()
    qbdf.loc[i, 'curSeasPassYdMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passYds'].min()
    qbdf.loc[i, 'curSeasPassYdMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passYds'].max()
    qbdf.loc[i, 'curSeasPassTdAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passTds'].mean()
    qbdf.loc[i, 'curSeasPassTdMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passTds'].min()
    qbdf.loc[i, 'curSeasPassTdMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passTds'].max()
    qbdf.loc[i, 'curSeasPassIntAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passInt'].mean()
    qbdf.loc[i, 'curSeasPassIntMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passInt'].min()
    qbdf.loc[i, 'curSeasPassIntMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passInt'].max()
    qbdf.loc[i, 'curSeasPassSackAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passSacked'].mean()
    qbdf.loc[i, 'curSeasPassSackMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passSacked'].min()
    qbdf.loc[i, 'curSeasPassSackMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passSacked'].max()
    qbdf.loc[i, 'curSeasPassLongAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passLong'].mean()
    qbdf.loc[i, 'curSeasPassLongMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passLong'].min()
    qbdf.loc[i, 'curSeasPassLongMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passLong'].max()
    qbdf.loc[i, 'curSeasPassRtngAvg'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passRating'].mean()
    qbdf.loc[i, 'curSeasPassRtngMin'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passRating'].min()
    qbdf.loc[i, 'curSeasPassRtngMax'] = qdf(d[soff], seas=[seas], week=range(1,week), plid=[plid])['passRating'].max()
    
    # opp deffense sack, int, and ffum - sum starting def players stats for each week
    defStarters = qdf(d[sdef], seas=[seas], week=[week], bsid=[bsid], team=[qbdf.loc[i, 'opponent']]).index.get_level_values('playerID').unique()
    if week==1:
        qbdf.loc[i, 'curSeasDefSackAvg'] = np.nan
        qbdf.loc[i, 'curSeasDefSackMin'] = np.nan
        qbdf.loc[i, 'curSeasDefSackMax'] = np.nan
        qbdf.loc[i, 'curSeasDefIntAvg'] = np.nan
        qbdf.loc[i, 'curSeasDefIntMin'] = np.nan
        qbdf.loc[i, 'curSeasDefIntMax'] = np.nan
        qbdf.loc[i, 'curSeasDefFFumAvg'] = np.nan
        qbdf.loc[i, 'curSeasDefFFumMin'] = np.nan
        qbdf.loc[i, 'curSeasDefFFumMax'] = np.nan
    else:
        qbdf.loc[i, 'curSeasDefSackAvg'] = np.mean([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['sacks'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefSackMin'] = min([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['sacks'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefSackMax'] = max([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['sacks'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefIntAvg'] = np.mean([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['def_int'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefIntMin'] = min([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['def_int'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefIntMax'] = max([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['def_int'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefFFumAvg'] = np.mean([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['fumbles_forced'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefFFumMin'] = min([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['fumbles_forced'].sum() for wk in range(1,week)])
        qbdf.loc[i, 'curSeasDefFFumMax'] = max([qdf(d[sdef], seas=[2010], week=[wk], plid=defStarters)['fumbles_forced'].sum() for wk in range(1,week)])
    
    # team stats for each week
    qbdf.loc[i, 'curSeasOffSackAvg'] = qdf(d[stm], seas=[seas], week=range(1,week), team=[team])['sacks'].mean()
    qbdf.loc[i, 'curSeasOffFirstDownsAvg'] = qdf(d[stm], seas=[seas], week=range(1,week), team=[team])['firstDowns'].mean()
    qbdf.loc[i, 'curSeasOffFourthDownAttAvg'] = qdf(d[stm], seas=[seas], week=range(1,week), team=[team])['fourthDownAtt'].mean()
    qbdf.loc[i, 'curSeasOffFourthDownConvAvg'] = qdf(d[stm], seas=[seas], week=range(1,week), team=[team])['fourthDownConv'].mean()

qbdf.head()

In [ ]:
# offense ff points calculator
ffPtsDict = {
    'ff_PassTds':        ( 4.0),
    'ff_PassYds':        ( 0.04),
    'ff_PassYdsOver300': ( 3.0),
    'ff_Int':            (-1.0),
    'ff_RushYds':        ( 0.1),
    'ff_RushTds':        ( 6.0),
    'ff_RushYdsOver100': ( 3.0),
    'ff_Fumble':         (-1.0),
    'ff_RecYds':         ( 0.1),
    'ff_Recs':           ( 1.0),
    'ff_RecTds':         ( 6.0),
    'ff_RecYdsOver100':  ( 3.0)
}

In [ ]:
B = np.array([ffPtsDict[stat] for stat in sorted(ffPtsDict.keys())])
X = qbdf[sorted(ffPtsDict.keys())].as_matrix()

qbdf['ff_PtsTot'] = np.dot(X, B)

In [ ]:
# drop rows with...
# missing fantasy points
qbdf = qbdf[np.isfinite(qbdf['ff_PtsTot'])]

# player did not play
qbdf = qbdf[(qbdf['didNotPlay']==False)]

In [ ]:
# drop stats used to calc fantasy points
# dropCols = [col for col in qbdf.columns if 'ff_' in col and col != 'ff_PtsTot']
dropCols = ['playerName', 'position', 'offense', 'startTime', 'didNotPlay']

qbdf.drop(dropCols, axis=1, inplace=True)

In [ ]:
qbdf.head()

In [ ]:
# output to csv
qbdf.to_csv('qb_starters_07_15.csv', index_share=False, index=True)

#### wide receivers

In [ ]:
d[tmrs].head()

In [ ]:
# adding features to quarterback df
wrdf = qdf(d[soff], seas=[2009, 2010, 2011, 2012, 2013, 2014, 2015]) #, week=[1])
wrdf = wrdf[wrdf.position == 'WR']

homeAwayOpp = {True:'away', False:'home'}

for i, r in qbdf.iterrows():
    # print i, x['home']
    #print 'seas={}, week={}, bsid={}, team={}, plid={}'.format(i[0], i[1], i[2], i[3], i[4])
    seas = i[0]
    week = i[1]
    bsid = i[2]
    team = i[3]
    plid = i[4]

In [ ]:
idx = pd.IndexSlice
groupCols = ['season', 'week', 'team']
d[soff].loc[idx[[2015],range(1,2),:,['den'],:],].groupby(level=[0,3,4])['recYds'].sum().rank(ascending=False).loc[idx[:,:,:],]

### dataframe to DMatrix

In [ ]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.1.0-posix-seh-rt_v5-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

In [ ]:
# import qbdf
qbdf = pd.read_csv('C:/Users/Phil/Google Drive/nfl/qb_starters_07_15.csv', low_memory=False)
qbdf.set_index(dfIn['starters'], inplace=True)
qbdf = qbdf.rename(columns = {'playerID.1':'playerID'})

In [ ]:
# qbdf.head()

In [ ]:
# print cont cols
for i, x in enumerate(qbdf.dtypes):
    if x.name=='float64' and not any(x in qbdf.columns[i] for x in ['Avg','Min','Max','ff_']):
        print qbdf.columns[i]#, x.name

In [ ]:
# print cat cols
for i, x in enumerate(qbdf.dtypes):
    if x.name != 'float64':
        print qbdf.columns[i]#, x.name

#### setup columns

In [ ]:
catCols = ['playerID','home','injuryStatus','opponent','weekday']

In [ ]:
contCols = [col for col in qbdf.columns if any(x in col for x in ['Avg','Min','Max'])]
contCols += ['pfrApproxValue', 'snapCountPct', 'windChill', 'windMPH', 'restDays']

In [ ]:
# trnCols = contCols + pd.get_dummies(qbdf[catCols]).columns.tolist()

In [ ]:
df_X = pd.merge(qbdf[contCols], pd.get_dummies(qbdf[catCols]), how='left', left_index=True, right_index=True)
df_y = qbdf['ff_PtsTot']

### run xgb

In [ ]:
grid = {
    'eta':[0.05], #[x / 100.0 for x in range(5, 25, 5)],
    'gamma':[x / 10.0 for x in range(3)],
    'max_depth':range(3, 10),
    'min_child_weight':range(1, 6),
    'subsample':[x / 10.0 for x in range(8,11)],
    'colsample_bytree':[x / 10.0 for x in range(8,11)]
}
grid

In [ ]:
# number of grid combincations
grid_n = 1
for key in grid.keys():
    grid_n *= len(grid[key])
print grid_n

In [ ]:
start = time.time()

# train/test on 2014/2015
qb_model_results = pd.DataFrame()

# setting hyper parameters
model_param = {
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'nthread':5,
    'silent':1
}

num_round = 151
model_results = {}

print 'Training...'
for i, gs in enumerate(itertools.islice(itertools.product(
    grid['eta'],
    grid['gamma'],
    grid['max_depth'],
    grid['min_child_weight'],
    grid['subsample'],
    grid['colsample_bytree'],
), grid_n)):
    tree_param = {
        'eta':gs[0],
        'gamma':gs[1],
        'max_depth':gs[2],
        'min_child_weight':gs[3],
        'subsample':gs[4],
        'colsample_bytree':gs[5]
    }
    
    param = model_param.copy()
    param.update(tree_param)
    
    for season in range(2013, 2015 + 1):
        for week in range(1, 21 + 1):
            print '\rGrid {}/{} - Season {} - Week {}'.format(i+1, grid_n, season, week),
            
            # setup X/y trn/tst
            X_trn = qdf(df_X, seas=range(2009,season))
            X_trn = X_trn.append(qdf(df_X, seas=[season], week=range(1,week)))
            X_tst = qdf(df_X, seas=[season], week=[week])
            
            y_trn = qdf(df_y, seas=range(2009,season))
            y_trn = y_trn.append(qdf(df_y, seas=[season], week=range(1,week)))
            y_tst = qdf(df_y, seas=[season], week=[week])
            
            # convert to DMatrix
            dtrn  = xgb.DMatrix(X_trn, label=y_trn)
            dtst  = xgb.DMatrix(X_tst, label=y_tst)
            
            # train model
            evallist  = [(dtrn,'train'), (dtst,'test')]
            bst = xgb.train( param, dtrn, num_round, evallist ,verbose_eval=None, evals_result=model_results)
            
            # export results to df
            week_results = pd.DataFrame(tree_param, index=[0])
            week_results['season'] = season
            week_results['week'] = week
            week_results['rmse_rows'] = len(y_tst)
            week_results = pd.concat([week_results] * num_round, ignore_index=True)
            week_results['round'] = range(len(week_results))
            week_results['rmse_test'] = pd.DataFrame(model_results['test'])
            week_results['rmse_train'] = pd.DataFrame(model_results['train'])
            
            # week_results = pd.merge(df_params, df_eval, left_index=True, right_index=True)
            qb_model_results = pd.concat([qb_model_results, week_results], ignore_index=True)

print '\nDone! Total run time: {}'.format(time.time() - start)

In [ ]:
# output to csv
qb_model_results.to_csv('qb_model_results_07_15.csv', index_share=False, index=True)

#### evaluate results

In [ ]:
param_cols = ['colsample_bytree','eta','gamma','max_depth','min_child_weight','subsample','round']

In [ ]:
testdf = qb_model_results.copy()

In [ ]:
def weighted_rmse(group):
    d = group['rmse_test']
    w = group['rmse_rows']
    return (d * w).sum() / w.sum()

In [ ]:
qb_model_results.groupby(param_cols).apply(weighted_rmse).sort_values()

In [ ]:
qb_model_results.groupby(param_cols).apply(weighted_rmse).sort_values().min()

In [ ]:
best_params = qb_model_results.groupby(param_cols).apply(weighted_rmse).sort_values().argmin()
best_params_dict = {}

for i, p in enumerate(param_cols):
    best_params_dict[p] = best_params[i]
    
best_params_dict

In [ ]:
# manually check params
best_params_dict = {
    'eta': .05,
    'gamma': 0.0,
    'max_depth': 3,
    'min_child_weight': 1,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'round': 129
}

#### train on all data

In [ ]:
dtrn  = xgb.DMatrix(df_X, label=df_y)

In [ ]:
model_param = {
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'nthread':5,
    'silent':1
}

param = best_params_dict.copy()
param.update(model_param)

In [ ]:
evallist  = [(dtrn, 'train')]
bst = xgb.train(param, dtrn, best_params_dict['round'], evallist ,verbose_eval=10)

In [ ]:
xgb.plot_importance(bst).get_figure().set_figheight(20)